In [ ]:
# A5 小書建立器（Jupyter 互動版 / 內嵌 [圖片%]、[左圖%][右圖%] / 不消耗上傳圖檔 / 封底無頁碼）
# 下載方案：FileLink + files/<name>.docx 備援、BytesIO 完整寫檔、檔名淨化 + 檔案資訊
# 需求：python-docx, pillow, ipywidgets
# 若缺套件，請先在新 cell 執行：
#   %pip install python-docx pillow ipywidgets

import io, uuid, re, hashlib
from pathlib import Path

# ===== 匯入套件與環境檢查 =====
missing = []
try:
    from docx import Document
    from docx.shared import Mm, Pt
    from docx.enum.section import WD_SECTION
    from docx.enum.text import WD_ALIGN_PARAGRAPH
    from docx.oxml import OxmlElement
    from docx.oxml.ns import qn
except Exception:
    missing.append("python-docx")

try:
    from PIL import Image
    _PIL_OK = True
except Exception:
    _PIL_OK = False
    missing.append("pillow (PIL)")

try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output, HTML, FileLink
except Exception:
    missing.append("ipywidgets / IPython")

if missing:
    try:
        from IPython.display import HTML as _HTML
        display(_HTML(f"""
        <div style="border:1px solid #f33;padding:12px;border-radius:8px">
          <b>缺少套件：</b> {', '.join(missing)}<br>
          請先在新 cell 執行：<code>%pip install python-docx pillow ipywidgets</code>
        </div>
        """))
    except Exception:
        pass

# ===== 版面與尺寸 =====
A5_W, A5_H = Mm(148), Mm(210)   # A5 直式
MARGIN = Mm(15)                 # 四邊 15 mm
USABLE_W_MM = 148 - 2 * 15      # 版心寬（mm）
USABLE_H_MM = 210 - 2 * 15      # 版心高（mm）

# 圖片最大尺寸（mm）— 預設較大，並由「圖片縮放 %」微調
IMG_MAX_W_MM_INNER = 118
IMG_MAX_H_MM_INNER = 120
IMG_MAX_W_MM_COVER = 118
IMG_MAX_H_MM_COVER = 140
IMG_MAX_W_MM_BACK  = 118
IMG_MAX_H_MM_BACK  = 140

# ===== 下載輔助：檔名淨化 + 檔案資訊 =====
SAFE_NAME_RE = re.compile(r'[^A-Za-z0-9._-]+')

def sanitize_filename(name: str, default="a5_book.docx") -> str:
    name = (name or "").strip()
    if not name:
        name = default
    if not name.lower().endswith(".docx"):
        name += ".docx"
    return SAFE_NAME_RE.sub("_", name) or default

def info_box(path: Path):
    try:
        size = path.stat().st_size
        sha = hashlib.sha256(path.read_bytes()).hexdigest()[:16]
        return HTML(f"<div style='color:#555'>大小：{size} bytes　SHA-256：<code>{sha}…</code></div>")
    except Exception:
        return HTML("<div style='color:#a55'>（無法讀取檔案大小/雜湊）</div>")

# ===== 基本工具 =====
def set_section_to_a5(section):
    section.page_width = A5_W
    section.page_height = A5_H
    section.left_margin = MARGIN
    section.right_margin = MARGIN
    section.top_margin = MARGIN
    section.bottom_margin = MARGIN

def add_page_number_footer(section, prefix="第 ", suffix=" 頁", start_at_1=True):
    """設定置中頁碼（只給內頁）。"""
    section.footer.is_linked_to_previous = False
    if start_at_1:
        sectPr = section._sectPr
        pgNumType = sectPr.find(qn('w:pgNumType'))
        if pgNumType is None:
            pgNumType = OxmlElement('w:pgNumType')
            sectPr.append(pgNumType)
        pgNumType.set(qn('w:start'), "1")

    ft = section.footer
    for p in list(ft.paragraphs):
        p._element.getparent().remove(p._element)

    p = ft.add_paragraph()
    p.alignment = WD_ALIGN_PARAGRAPH.CENTER

    if prefix:
        p.add_run(prefix)

    fldChar1 = OxmlElement('w:fldChar'); fldChar1.set(qn('w:fldCharType'), 'begin')
    instrText = OxmlElement('w:instrText'); instrText.set(qn('xml:space'), 'preserve'); instrText.text = ' PAGE '
    fldChar2 = OxmlElement('w:fldChar'); fldChar2.set(qn('w:fldCharType'), 'separate')
    fldText  = OxmlElement('w:t'); fldText.text = "1"
    fldChar3 = OxmlElement('w:fldChar'); fldChar3.set(qn('w:fldCharType'), 'end')

    r = p.add_run()
    r._r.append(fldChar1); r._r.append(instrText); r._r.append(fldChar2); r._r.append(fldText); r._r.append(fldChar3)

    if suffix:
        p.add_run(suffix)

def _extract_name_and_bytes(record):
    """從 FileUpload 單一記錄抽取 (name, bytes)；相容 content/data 與 metadata/name 欄位"""
    if not isinstance(record, dict):
        return (None, None)
    meta = record.get('metadata') or {}
    name = meta.get('name') or record.get('name')
    content = record.get('content')
    if content is None:
        content = record.get('data')
    return (name, content)

def fileupload_get_list(uploader):
    """回傳 [(filename, bytes), ...]，不改變 uploader.value。"""
    v = uploader.value
    items = []
    if not v:
        return items
    if isinstance(v, (tuple, list)):
        for rec in v:
            name, blob = _extract_name_and_bytes(rec)
            if name and blob is not None:
                items.append((name, blob))
    elif isinstance(v, dict):
        for _, rec in v.items():
            name, blob = _extract_name_and_bytes(rec)
            if name and blob is not None:
                items.append((name, blob))
    return items

# ===== 圖片計算與插入 =====
def compute_scaled_width_mm(image_bytes, max_w_mm, max_h_mm, scale=1.0):
    """計算在最大寬/高與倍率下的等比例寬度（mm）。"""
    max_w_mm = max_w_mm * scale
    max_h_mm = max_h_mm * scale
    width_mm_to_use = max_w_mm
    if _PIL_OK:
        try:
            with Image.open(io.BytesIO(image_bytes)) as im:
                w, h = im.size
                if w > 0 and h > 0:
                    r = w / h
                    width_by_height_cap = max_h_mm * r
                    width_mm_to_use = min(max_w_mm, width_by_height_cap)
        except Exception:
            width_mm_to_use = max_w_mm
    return width_mm_to_use

def add_picture_to_paragraph(paragraph, image_bytes, width_mm, align_center=True):
    run = paragraph.add_run()
    run.add_picture(io.BytesIO(image_bytes), width=Mm(width_mm))
    if align_center:
        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

def add_single_image_block(doc, image_bytes, max_w_mm, max_h_mm, scale=1.0, align_center=True):
    width_mm = compute_scaled_width_mm(image_bytes, max_w_mm, max_h_mm, scale=scale)
    p = doc.add_paragraph()
    add_picture_to_paragraph(p, image_bytes, width_mm, align_center=align_center)

def add_two_images_row(doc, left_bytes, right_bytes, each_max_w_mm, max_h_mm,
                       left_scale=1.0, right_scale=1.0, gap_mm=6):
    """用 2 欄表格讓兩張圖並排（無邊框；靠圖片寬度保留中間留白）。"""
    tbl = doc.add_table(rows=1, cols=2)
    tbl.autofit = True
    # 左
    if left_bytes:
        w_left_mm = compute_scaled_width_mm(left_bytes, each_max_w_mm, max_h_mm, scale=left_scale)
        pL = tbl.rows[0].cells[0].paragraphs[0]
        add_picture_to_paragraph(pL, left_bytes, w_left_mm, align_center=True)
    # 右
    if right_bytes:
        w_right_mm = compute_scaled_width_mm(right_bytes, each_max_w_mm, max_h_mm, scale=right_scale)
        pR = tbl.rows[0].cells[1].paragraphs[0]
        add_picture_to_paragraph(pR, right_bytes, w_right_mm, align_center=True)

# ===== 標記解析與渲染 =====
_marker_pattern = re.compile(r'(\[(?:圖片|左圖|右圖)(?:\d+)?%?\])')

def parse_marker(token):
    """
    從標記字串取出型態與百分比倍率。
    e.g. "[圖片60%]" → ("single", 0.6)
         "[左圖80%]" → ("left", 0.8)
         "[右圖]"    → ("right", 1.0)
    """
    m = re.match(r'^\[(圖片|左圖|右圖)(\d+)?%?\]$', token)
    if not m:
        return (None, None)
    kind = m.group(1)
    perc = float(m.group(2)) / 100.0 if m.group(2) else 1.0
    if kind == "圖片":
        return ("single", perc)
    elif kind == "左圖":
        return ("left", perc)
    elif kind == "右圖":
        return ("right", perc)
    return (None, None)

def render_marked_block(doc, raw_text, images, start_idx,
                        font_size_pt, max_w_mm, max_h_mm, base_scale=1.0):
    """
    解析 raw_text 中的 [圖片] / [圖片60%] / [左圖%][右圖%]，
    依序使用 images[start_idx:]，回傳新的 index。
    若沒有任何標記且仍有可用圖片，預設在文字後加一張圖。
    """
    idx = start_idx
    used_marker = False
    buffer_left = None  # (bytes, left_local_scale)

    tokens = _marker_pattern.split(str(raw_text))

    for tk in tokens:
        if not tk:
            continue

        kind, local = parse_marker(tk)
        if kind == "single":
            used_marker = True
            if idx < len(images):
                _, ibytes = images[idx]
                eff_scale = base_scale * local
                add_single_image_block(doc, ibytes, max_w_mm, max_h_mm,
                                       scale=eff_scale, align_center=True)
                idx += 1

        elif kind in ("left", "right"):
            used_marker = True
            if kind == "left":
                # 暫存左圖，等待右圖
                if idx < len(images):
                    buffer_left = (images[idx][1], base_scale * local)
                    idx += 1
                else:
                    buffer_left = (None, base_scale * local)
            elif kind == "right":
                # 成對輸出左右圖
                left_bytes, left_scale = buffer_left if buffer_left else (None, base_scale)
                right_bytes = None
                right_scale = base_scale * local
                if idx < len(images):
                    right_bytes = images[idx][1]
                    idx += 1

                each_max_w = (max_w_mm - 6) / 2.0  # 預留一點間隔
                add_two_images_row(
                    doc,
                    left_bytes,
                    right_bytes,
                    each_max_w, max_h_mm,
                    left_scale=left_scale,
                    right_scale=right_scale,
                    gap_mm=6
                )
                buffer_left = None

        else:
            # 純文字
            text_block = tk.strip()
            if text_block:
                for para in text_block.split("\n\n"):
                    p = doc.add_paragraph(para.strip())
                    p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
                    p.paragraph_format.space_after = Pt(6)
                    for run in p.runs:
                        run.font.size = Pt(font_size_pt)

    # 若結束仍有未配對的 [左圖]，視為單圖插入
    if buffer_left and buffer_left[0] is not None:
        add_single_image_block(doc, buffer_left[0], max_w_mm, max_h_mm,
                               scale=buffer_left[1], align_center=True)
        buffer_left = None
        used_marker = True

    # 若沒用過任一標記，且仍有可用圖片，預設文字後放一張圖（用 base_scale）
    if (not used_marker) and (idx < len(images)):
        _, ibytes = images[idx]
        add_single_image_block(doc, ibytes, max_w_mm, max_h_mm,
                               scale=base_scale, align_center=True)
        idx += 1

    return idx

def render_page(doc, raw_text, images, start_idx, font_size_pt, max_w_mm, max_h_mm,
                base_scale=1.0, pagebreak_after=True):
    """渲染一整頁（文字+圖），回傳新的 image index。"""
    text_present = bool(str(raw_text).strip())
    idx = start_idx
    if text_present or idx < len(images):
        idx = render_marked_block(doc, str(raw_text), images, idx, font_size_pt,
                                  max_w_mm, max_h_mm, base_scale=base_scale)
        if pagebreak_after:
            doc.add_page_break()
    return idx

# ===== UI =====
title = widgets.HTML("<h3>A5 小書 Word 產生器（[圖片%] / [左圖%][右圖%]；封底無頁碼；圖片可重用）</h3>")

# 封面（可多圖，便於並排）
cover_text = widgets.Textarea(
    value="我的小書\n副標題\n\n[圖片]\n也可用 [左圖80%][右圖50%]",
    description="封面文字",
    placeholder="可在文字中放入 [圖片] 或 [圖片60%]、[左圖%][右圖%]",
    layout=widgets.Layout(width="100%", height="140px")
)
cover_imgs = widgets.FileUpload(accept="image/*", multiple=True, description="封面圖片（可多檔）")

# 內頁：以兩個換行分頁；段內可用標記
inner_texts = widgets.Textarea(
    value="第一章開場……\n\n第二章轉折……\n\n第三章：並排示例\n[左圖80%][右圖60%]",
    description="內頁文字",
    placeholder="每兩個換行分隔一頁；段內可用 [圖片] / [圖片60%] / [左圖%][右圖%]",
    layout=widgets.Layout(width="100%", height="220px")
)
inner_imgs = widgets.FileUpload(accept="image/*", multiple=True, description="內頁圖片（依序使用，不會被消耗）")

# 封底（可多圖）
back_text = widgets.Textarea(
    value="封底簡介……\n\n[圖片60%]",
    description="封底文字",
    layout=widgets.Layout(width="100%", height="140px")
)
back_imgs = widgets.FileUpload(accept="image/*", multiple=True, description="封底圖片（可多檔）")

# 選項
font_size = widgets.IntSlider(value=12, min=9, max=16, step=1, description="字級")
img_scale = widgets.FloatSlider(value=1.0, min=0.6, max=2.0, step=0.05, description="圖片縮放（全域倍率）")
img_scale.style.handle_color = 'lightblue'
img_scale.readout_format = '.2f'
outfile_name = widgets.Text(value=f"a5_book_{uuid.uuid4().hex[:6]}.docx", description="輸出檔名")
add_numbers = widgets.Checkbox(value=True, description="內頁加頁碼（封面/封底不加）")

build_btn = widgets.Button(description="生成 A5 小書 (.docx)", button_style="success", icon="file")
log = widgets.Output()

ui = widgets.VBox([
    title,
    widgets.HTML("<b>封面</b>"),
    cover_text, cover_imgs,
    widgets.HTML("<hr>"),
    widgets.HTML("<b>內頁（每兩個換行分頁；段內可用標記控制圖位與比例）</b>"),
    inner_texts, inner_imgs,
    widgets.HTML("<hr>"),
    widgets.HTML("<b>封底</b>"),
    back_text, back_imgs,
    widgets.HTML("<hr>"),
    widgets.HBox([font_size, img_scale, add_numbers]),
    outfile_name,
    build_btn,
    log
])

def build_doc(_):
    with log:
        clear_output()
        # —— 安全檔名 + 工作目錄 ——
        safe_name = sanitize_filename(outfile_name.value, default="a5_book.docx")
        outpath = Path.cwd() / safe_name
        doc = Document()

        # ===== 封面（不加頁碼）=====
        set_section_to_a5(doc.sections[0])
        cover_images = fileupload_get_list(cover_imgs)
        cov_idx = 0
        cov_idx = render_page(
            doc,
            raw_text=cover_text.value,
            images=cover_images,
            start_idx=cov_idx,
            font_size_pt=font_size.value + 2,
            max_w_mm=IMG_MAX_W_MM_COVER,
            max_h_mm=IMG_MAX_H_MM_COVER,
            base_scale=float(img_scale.value),
            pagebreak_after=False
        )

        # ===== 內頁（新節；可加頁碼）=====
        inner_section = doc.add_section(WD_SECTION.NEW_PAGE)
        set_section_to_a5(inner_section)
        if add_numbers.value:
            add_page_number_footer(inner_section, prefix="第 ", suffix=" 頁", start_at_1=True)

        inner_images = fileupload_get_list(inner_imgs)
        idx = 0
        text_pages = [t.strip() for t in inner_texts.value.split("\n\n") if t.strip()] or [""]
        for page_text in text_pages:
            idx = render_page(
                doc,
                raw_text=page_text,
                images=inner_images,
                start_idx=idx,
                font_size_pt=font_size.value,
                max_w_mm=IMG_MAX_W_MM_INNER,
                max_h_mm=IMG_MAX_H_MM_INNER,
                base_scale=float(img_scale.value),
                pagebreak_after=True
            )

        # ===== 封底（新節；不加頁碼、不承襲內頁 footer）=====
        back_section = doc.add_section(WD_SECTION.NEW_PAGE)
        set_section_to_a5(back_section)
        back_section.footer.is_linked_to_previous = False
        for p in list(back_section.footer.paragraphs):
            p._element.getparent().remove(p._element)

        back_images = fileupload_get_list(back_imgs)
        back_idx = 0
        back_idx = render_page(
            doc,
            raw_text=back_text.value,
            images=back_images,
            start_idx=back_idx,
            font_size_pt=font_size.value,
            max_w_mm=IMG_MAX_W_MM_BACK,
            max_h_mm=IMG_MAX_H_MM_BACK,
            base_scale=float(img_scale.value),
            pagebreak_after=False
        )

        # ===== 存檔（BytesIO -> write_bytes，避免半寫入）=====
        buf = io.BytesIO()
        doc.save(buf)                       # 完整寫入記憶體（docx 是 zip）
        outpath.write_bytes(buf.getvalue()) # 一次性落盤

        # 訊息 + 下載 + 檔案資訊
        display(HTML(f"""
        <div style="border:1px solid #0a0;padding:12px;border-radius:8px;margin-bottom:8px">
          <b>完成！</b> 已輸出：<code>{outpath.name}</code>
        </div>
        """))
        display(FileLink(str(outpath), result_html_prefix="🔗 下載： "))
        display(HTML(f'<div style="color:#555">備援連結：<a href="files/{outpath.name}" download>files/{outpath.name}</a></div>'))
        display(info_box(outpath))

        # （可選）Colab 自動觸發下載；其他環境不會報錯
        try:
            from google.colab import files as _colab_files
            _colab_files.download(str(outpath))
        except Exception:
            pass

build_btn.on_click(build_doc)
display(ui)